In [171]:
from manim import *
config.media_embed = True
config.media_width = "50%"
config.verbosity = "WARNING"

In [ ]:
%%manim -v WARNING -qh --disable_caching Wave
import numpy as np
from random import *


class Wave(Scene):
    def construct(self):

        maze_string = """
#######################################################
#  #################            ##                    #
# ##################           ####                   #
# #################            ####                   #
#  ###############             #####               # ##
#      #########               #####               ####
#         ###                  ######            ######
#          ###            ##   #####    ###       #####
#          ####      ########   ####  #####        ## #
#          #####   ##########   ###  ########       # #
#         #####   ###########        ########         #
#         ####   ###########        ##########        #
#        ##      ###########        ##########        #
#      ####     ############      #############       #
#    ######     ############     #############        #
# #########  ## ###########     #########    #        #
# ############### #########     #######               #
# ###############   ######      ######                #
# ###############    #####       ####                 #
#   #############      #                ##            #
#     #  #######                       ########### ####
#          ###         #              #################
# ##                  ####            #################
#####                ######          ##################
######                ######         ##################
# ###      ###        #######  ###   ###############  #
#         ####         ############   ####  #######   #
#        #####          ############          ###     #
#         ###            ##########                   #
#######################################################
"""

        maze = []  # 2D array of squares like we see it
        maze_bool = []  # 2D array of true/false values
        all_squares = VGroup()

        for row in maze_string.strip().split("\n"):
            maze.append([])
            maze_bool.append([])
            for char in row:
                maze[-1].append(
                    Square(
                        color=WHITE if char == " " else BLACK,
                        fill_opacity=1
                    ).scale(0.1)
                )
                maze_bool[-1].append(char == " ")
                all_squares.add(maze[-1][-1])
        
        rows = len(maze)
        cols = len(maze[0])
        
        self.play(FadeIn(all_squares.arrange_in_grid(rows, cols, buff=0.05)))

        x, y = 1, 1
        distances = {(x, y): 0}
        stack = [(x, y, 0)]

        while stack:
            x, y, d = stack.pop(0)
            for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
                if x + dx < 0 or x + dx >= cols or y + dy < 0 or y + dy >= rows:
                    continue
                if maze_bool[y + dy][x + dx] and (x + dx, y + dy) not in distances:
                    distances[(x + dx, y + dy)] = d + 1
                    stack.append((x + dx, y + dy, d + 1))
        
        max_distance = max(distances.values())
        colors = ["#ef476f", "#ffd166", "#06d6a0", "#118ab2"]
        all_colors = color_gradient(colors, max_distance + 1)

        group = []
        for c in range(max_distance + 1):
            group.append(
                AnimationGroup(
                    *[
                        maze[y][x].animate.set_color(color=all_colors[c])
                        for x, y in distances.keys()
                        if distances[(x, y)] == c
                    ]
                )
            )
            
        self.play(AnimationGroup(*group, lag_ratio=0.01))
    

    



Manim Community v0.18.1

In [106]:
%%manim -v WARNING -qh --disable_caching Hilbert

class Hilbert(Scene):

    def construct(self):
        def unit_hilbert_curve(points):
            return VMobject(color=WHITE).set_points_as_corners(points)
        
        points = [LEFT+DOWN, LEFT+UP, RIGHT+UP, RIGHT+DOWN]
        hilbert = unit_hilbert_curve(points).scale(3)
        hilbert_points = list(hilbert.get_start_anchors()) + [hilbert.get_end_anchors()[-1]]
        self.play(Create(hilbert))

        dl = unit_hilbert_curve(points)
        ul = unit_hilbert_curve(points)
        ur = unit_hilbert_curve(points)
        dr = unit_hilbert_curve(points)
        
        self.play(Create(ur), Create(ul), Create(dr), Create(dl))

        self.play(
            AnimationGroup(
                dl.animate.align_to(hilbert, hilbert_points[0]).set_color(DARK_GRAY).rotate(-PI/2),
                ul.animate.align_to(hilbert, hilbert_points[1]).set_color(DARK_GRAY),
                ur.animate.align_to(hilbert, hilbert_points[2]).set_color(DARK_GRAY), 
                dr.animate.align_to(hilbert, hilbert_points[3]).set_color(DARK_GRAY).rotate(PI/2),
            ),
            FadeOut(hilbert)
        )
        hilbert_points = [dl.get_end_anchors()[-1]] + list(dl.get_start_anchors())[::-1] + list(ul.get_start_anchors()) + [ul.get_end_anchors()[-1]] + list(ur.get_start_anchors()) + [ur.get_end_anchors()[-1]] + [dr.get_end_anchors()[-1]] + list(dr.get_start_anchors())[::-1]

        new_hilbert = unit_hilbert_curve(hilbert_points)
        self.play(Create(new_hilbert))


        

Manim Community v0.18.1

In [ ]:
%%manim -v WARNING -qh --disable_caching BasicAnimation 
import numpy as np

class BasicAnimation(Scene):
    def construct(self):
        self.next_section("Rate functions")
        polys = VGroup(
            *[
                RegularPolygon(5, radius=1, fill_opacity=0.5, color=WHITE) for _ in range(5)
            ]
        ).arrange(RIGHT)
        self.play(DrawBorderThenFill(polys), run_time=2)
        self.play(
            Rotate(polys[0], angle=PI, rate_func=lambda t: t),
            Rotate(polys[1], angle=PI, rate_func=lambda t: np.sin(t * PI)),
            Rotate(polys[2], angle=PI, rate_func=smooth),
            Rotate(polys[3], angle=PI, rate_func=there_and_back),
            Rotate(polys[4], angle=PI, rate_func=lambda t: 1 - abs(1 - 2 * t)),
            run_time=2
        )
        self.wait(1)
        


In [ ]:
#more rate functions on https://easings.net/
%%manim -v WARNING -qh --disable_caching RateFunctions2

class RateFunctions2(Scene):
    def construct(self):
        lines = VGroup(
            Line(LEFT*3, RIGHT).set_color(RED),
            Line(LEFT*3, RIGHT).set_color(GREEN),
            Line(LEFT*3, RIGHT).set_color(BLUE),
            Line(LEFT*3, RIGHT).set_color(YELLOW)
        )
        lines.arrange(DOWN, buff=1)
        lines.shift(LEFT)

        labels = VGroup(
            Tex(r"smooth (default)"),
            Tex(r"linear"),
            Tex(r"there\_and\_back"),
            Tex(r"rush\_into")
        )
        for label, line in zip(labels, lines):
            label.next_to(line, RIGHT, buff=1)

        dots = VGroup(
            Dot(color=RED),
            Dot(color=GREEN),
            Dot(color=BLUE),
            Dot(color=YELLOW)
        )
        for dot, line in zip(dots, lines):
            dot.move_to(line.point_from_proportion(0))

        self.play(Create(lines), FadeIn(labels))
        self.play(FadeIn(dots))
        group =[]
        for dot, line, rate_functions in zip(dots, lines, [smooth, linear, there_and_back, rush_into]):
            group.append(MoveAlongPath(dot, line, rate_func=rate_functions))

        self.play(AnimationGroup(*group, lag_ratio=0, run_time=3))

Manim Community v0.18.1

In [125]:
%%manim -v WARNING -qh --disable_caching TargetUsage 
import numpy as np

class TargetUsage(Scene):
    def construct(self):
        c = Circle()
        self.play(Create(c))
        c.generate_target()
        c.target.shift(LEFT)
        c.target.scale(2)
        self.play(MoveToTarget(c))

Manim Community v0.18.1

In [127]:
%%manim -v WARNING -qh --disable_caching CustomAnimation
import numpy as np

class CustomAnimation(Scene):
    def construct(self):
        def spiral_out(mobj, t): # t is in [0, 1] and t is a completion ratio
            radius = 4 * t
            angle = 2 * PI * t * 2
            mobj.move_to(radius * np.array([np.cos(angle), np.sin(angle), 0]))
            color = interpolate_color(RED, BLUE, t)
            mobj.set_color(color)
        
        c = Circle()
        self.play(Create(c))
        self.play(UpdateFromAlphaFunc(c, spiral_out), run_time=2)

Manim Community v0.18.1

In [133]:
%%manim -v WARNING -qh --disable_caching CustomAnimation
import numpy as np

class Disperse(Animation):
    def __init__(self, mobject, dot_radius=0.05, dot_number=100, **kwargs):
        super().__init__(mobject, **kwargs)
        self.dot_radius = dot_radius
        self.dot_number = dot_number
    
    def begin(self):
        dots = VGroup(
            *[
                Dot(radius=self.dot_radius).move_to(self.mobject.point_from_proportion(p))
                for p in np.linspace(0, 1, self.dot_number)
            ]
        )
        for dot in dots:
            dot.initial_position = dot.get_center()
            dot.shift_vector = (dot.get_center() - self.mobject.get_center())
        dots.set_oppacity(0)
        self.mobject.add(dots)
        self.dots = dots
        super().begin()

    def clean_up_from_scene(self, scene: "Scene") -> None:
        super().clean_up_from_scene(scene)
        self.mobject.remove(self.dots)

    def interpolate_mobject(self, alpha: float) -> None:
        if alpha <= 0.5:
            self.mobject.set_opacity(1 - 2 * alpha, family=False)
            self.dots.set_opacity(2 * alpha)
        else:
            self.mobject.set_opacity(0)
            self.dots.set_opacity(2*(1-alpha))
            for dot in self.dots:
                dot.move_to(dot.initial_position + 2*(alpha-0.5) * dot.shift_vector)

class CustomAnimation(Scene):
    def construct(self):
        c = Circle(color=WHITE, fill_opacity=1).scale(2)
        self.play(Write(c))
        self.play(Disperse(c, dot_number=200), run_time=2)

Manim Community v0.18.1

<string>:19: DeprecationWarning: This method is not guaranteed to stay around. Please prefer setting the attribute normally or with Mobject.set().


In [ ]:
%%manim -v WARNING -qh --disable_caching GraohExample 
import numpy as np

class GraohExample(Scene):
    def construct(self):
        # the graph class expects a list of vertices and edges
        vertices = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
        edges = [(1, 2), (2, 3), (3, 4), (2, 4), (2, 5), (6, 5),
                 (1, 7), (5, 7), (2, 8), (1, 9), (10, 8), (5, 11)]

        # we're using the layout_config's seed parameter to deterministically set the
        # vertex positions (it is otherwise set randomly)
        g = Graph(vertices, edges, layout_config={"seed": 0}).scale(1)
        g_ram = Graph(vertices, edges).scale(1)
        g_ram_2 = Graph(vertices, edges).scale(1)
        self.play(Create(VGroup(g, g_ram, g_ram_2).arrange(RIGHT)), run_time=2)
        self.wait(1)

        # the graph contains updaters that align edges with their vertices
        self.play(g.vertices[6].animate.shift((LEFT + DOWN) * 0.5))

        self.remove(*self.mobjects)
        # the graphs can also contain labels and be organized into specific layouts
        # (see the Graph class documentation for the list of all possible layouts)
        h = Graph(vertices, edges, labels=True, layout="circular")

        self.play(Write(h))
        self.play(FadeOut(h))

        self.play(Create(g))

        # color the vertex 5 and all of its neighbours
        v = 5
        self.play(
            Flash(g.vertices[v], color=RED, flash_radius=0.5),
            g.vertices[v].animate.set_color(RED),
            *[g.edges[e].animate.set_color(RED) for e in g.edges if v in e],
        )


Manim Community v0.18.1

In [162]:
%%manim -v WARNING -qh --disable_caching ExoticGraph 
import numpy as np
from random import uniform, randint, seed

class ExoticGraph(Scene):
    def construct(self):
        seed(0xDEADBEEF)
        vertices = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
        edges = [(1, 2), (2, 3), (3, 4), (2, 4), (2, 5), (6, 5),
                    (1, 7), (5, 7), (2, 8), (1, 9), (10, 8), (5, 11)]
        
        def RandomStar():
            """Create a pretty random star."""
            return Star(
                randint(5, 7),
                fill_opacity=1,
                outer_radius=0.1,
                color=WHITE).rotate(uniform(0, 2 * PI))

        def RandomSkyLine(u, v, z_index=None):
            """Create a pretty random sky line. The z_index is necessary, since it is
            passed by the graph constructor to edges so they're behind vertices."""
            return DashedLine(u, v, dash_length=uniform(0.03, 0.07), z_index=z_index)

        g = Graph(vertices, edges,
            layout_config={"seed": 0},
            vertex_type=RandomStar,
            edge_type=RandomSkyLine,
        ).scale(2).rotate(-PI / 2)

        self.play(Write(g))

        self.play(FadeOut(g))

        

Manim Community v0.18.1

In [163]:
%%manim -v WARNING -qh --disable_caching GraphGenerationExample
from random import *
import networkx as nx


class GraphGenerationExample(Scene):
    def construct(self):
        seed(0xDEADBEEF)

        n = 12     # number of vertices
        p = 3 / n  # probability that there is an edge between a pair

        # generate until our graph is not connected (so it looks nicer)
        graph = None
        while graph is None or not nx.is_connected(graph):
            graph = nx.generators.random_graphs.gnp_random_graph(n, p)

        g = Graph.from_networkx(graph, layout_config={"seed": 0}).scale(2.2).rotate(-PI / 2)

        self.play(Write(g))

Manim Community v0.18.1

In [ ]:
%%manim -v WARNING -qh --disable_caching MovingCameraExample

class MovingCameraExample(MovingCameraScene):
    def construct(self):
        square = Square()

        self.play(Write(square))

        self.camera.frame.save_state()

        # zoom for the square to fill in the entire view (+ a bit of space)
        self.play(self.camera.frame.animate.set_height(square.height * 1.5))

        circle = Circle().next_to(square, LEFT)
        self.play(
            AnimationGroup(
                self.camera.frame.animate.move_to(circle),
                Write(circle),
                lag_ratio=0.5
            )
        )

        # zoom out (increasing frame size covers more of the screen)
        self.play(self.camera.frame.animate.scale(1.3))

        triangle = Triangle().next_to(square, RIGHT)

        # move the camera again
        self.play(
            AnimationGroup(
                self.camera.frame.animate.move_to(triangle),
                Write(triangle),
                lag_ratio=0.5,
            )
        )

        self.wait(0.5)

        self.play(self.camera.frame.animate.restore())




Manim Community v0.18.1

In [175]:
%%manim -v WARNING -qh --disable_caching CameraBackgroundChanging

class CameraBackgroundChanging(MovingCameraScene):
    def construct(self):
        self.camera.background_color = WHITE
        self.camera.frame.scale(0.6)
        square = Square(color=BLACK)
        self.play(Create(square))
        self.play(self.camera.frame.animate.scale(2))

Manim Community v0.18.1

In [183]:
%%manim -v WARNING -qh --disable_caching DotMovingAlongCircle

class DotMovingAlongCircle(MovingCameraScene):
    def construct(self):
        dot = Dot(radius=0.2, color=RED)
        circle_path = Circle(radius=2, color=WHITE)
        dot.move_to(circle_path.point_from_proportion(0))
        self.play(Create(circle_path), Create(dot))

        self.camera.frame.save_state()
        self.play(self.camera.frame.animate.move_to(dot).scale(0.2))

        # define a updater for camera frame
        def update_frame(frame):
            frame.move_to(dot)

        self.camera.frame.add_updater(update_frame)
        self.play(MoveAlongPath(dot, circle_path), run_time=4, rate_func=there_and_back)


Manim Community v0.18.1